# Setup

## Instalação das bibliotecas

In [ ]:
%pip install pandas numpy matplotlib gdown networkx seaborn scikit-learn scipy statsmodels

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 4.3 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.7 MB 4.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/10.7 MB 4.0 MB/s eta 0:00:03
   ------- -----------

## Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from ChartGenerator import ChartGenerator
import seaborn as sns
from scipy.stats import chi2_contingency, ttest_ind, zscore as _zscore

# Download do Dataframe

In [ ]:
nome_arquivo = ''

In [ ]:
!gdown [link]

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=[link]

but Gdown can't. Please check connections and permissions.


In [ ]:
df = pd.read_csv(nome_arquivo)
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: ''

# Instruções

# Análise Estatistica

In [ ]:
def estatistica_descritiva_numerica(df):
    """
    Gera estatísticas descritivas e plot de histograma e boxplot
    para todas as colunas numéricas do DataFrame.
    """
    num_cols = df.select_dtypes(include=np.number).columns
    stats = df[num_cols].describe().T
    display(stats)  # exibe tabela de estatísticas
    
    for col in num_cols:
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        sns.histplot(df[col], ax=axes[0], kde=True, color='skyblue')
        axes[0].set_title(f'Histograma de {col}')
        
        sns.boxplot(x=df[col], ax=axes[1], color='lightgreen')
        axes[1].set_title(f'Boxplot de {col}')
        
        plt.tight_layout()
        plt.show()

def estatistica_descritiva_categorica(df):
    """
    Gera estatísticas descritivas (contagem, %)
    e plot de barras para todas as colunas categóricas do DataFrame.
    """
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        counts = df[col].value_counts()
        pct = df[col].value_counts(normalize=True).mul(100).round(2)
        summary = pd.DataFrame({'count': counts, 'percent (%)': pct})
        display(summary)
        
        plt.figure(figsize=(8, 4))
        sns.countplot(data=df, x=col, order=counts.index, palette='pastel')
        plt.title(f'Distribuição de {col}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

# Matriz de Markov

In [ ]:
def criar_matriz_de_markov(df, coluna_estado):
    """
    Constrói a matriz de transição de Markov a partir de uma coluna de estados
    em ordem temporal (linhas consecutivas).
    Retorna um DataFrame onde cada linha é o estado anterior e cada coluna o estado seguinte,
    preenchido com probabilidades de transição.
    """
    estados = df[coluna_estado]
    anterior = estados.shift(1)
    atual = estados
    contagens = pd.crosstab(anterior, atual) \
                  .drop(index=[np.nan], errors='ignore') \
                  .fillna(0)
    matriz = contagens.div(contagens.sum(axis=1), axis=0)
    return matriz

def plotar_grafo_markov(matriz, limiar=0.0, layout='spring', figsize=(8, 6)):
    """
    Recebe uma matriz de Markov (DataFrame de probabilidades) e plota
    um grafo direcionado com pesos proporcionais às probabilidades.
     - limiar: probabilidade mínima para exibir a aresta
     - layout: 'spring', 'circular' ou 'shell'
    """
    G = nx.DiGraph()
    for origem in matriz.index:
        for destino in matriz.columns:
            p = matriz.at[origem, destino]
            if p > limiar:
                G.add_edge(origem, destino, weight=p)

    if G.number_of_edges() == 0:
        print("Não há transições acima do limiar especificado.")
        return

    plt.figure(figsize=figsize)
    if layout == 'circular':
        pos = nx.circular_layout(G)
    elif layout == 'shell':
        pos = nx.shell_layout(G)
    else:
        pos = nx.spring_layout(G)

    pesos = nx.get_edge_attributes(G, 'weight')
    # desenha nós e rótulos
    nx.draw_networkx_nodes(G, pos, node_size=1500, node_color='lightblue')
    nx.draw_networkx_labels(G, pos, font_size=10)
    # desenha arestas com largura proporcional à probabilidade
    larguras = [v * 5 for v in pesos.values()]
    nx.draw_networkx_edges(
        G, pos,
        arrowstyle='->',
        arrowsize=12,
        width=larguras,
        edge_color='gray'
    )
    # escreve os valores de probabilidade nas arestas
    etiquetas = {k: f"{v:.2f}" for k, v in pesos.items()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=etiquetas, font_size=8)

    plt.axis('off')
    plt.show()

# Monte Carlo

In [ ]:
def monte_carlo(func, distributions, n_sim=1000, summary=True, plot=True, bins=30):
    """
    Gera simulação de Monte Carlo genérica.
    
    Parâmetros:
    - func: função a ser simulada; deve aceitar argumentos nomeados.
    - distributions: dict onde cada chave é nome de argumento de func e o valor é
      um callable que retorna uma amostra aleatória para esse argumento.
    - n_sim: número de iterações da simulação.
    - summary: se True, exibe estatísticas descritivas dos resultados.
    - plot: se True, plota histograma dos resultados (válido para saída única).
    - bins: número de bins do histograma.
    
    Retorno:
    DataFrame com os resultados da simulação.
    """
    results = []
    for _ in range(n_sim):
        # amostra parâmetros
        params = {name: samp() for name, samp in distributions.items()}
        out = func(**params)
        # organiza saída
        if isinstance(out, dict):
            results.append(out)
        elif isinstance(out, (tuple, list)):
            results.append({f'output_{i}': v for i, v in enumerate(out)})
        else:
            results.append({'result': out})
    df_sim = pd.DataFrame(results)
    
    if summary:
        print(df_sim.describe().T)
    if plot and df_sim.shape[1] == 1:
        col = df_sim.columns[0]
        plt.figure(figsize=(8, 4))
        sns.histplot(df_sim[col], bins=bins, kde=True, color='skyblue')
        plt.title(f'Distribuição de {col}')
        plt.tight_layout()
        plt.show()
    
    return df_sim

# Avaliação dos resultados

In [ ]:
def teste_chi_quadrado(df, col1, col2, alpha=0.05):
    """
    Teste de independência qui-quadrado entre duas variáveis categóricas.
    Retorna estatística, p-valor, graus de liberdade, frequências esperadas e decisão.
    """
    tabela = pd.crosstab(df[col1], df[col2])
    chi2, p, dof, expected = chi2_contingency(tabela)
    decisao = "rejeita H0 (não-independentes)" if p < alpha else "falha em rejeitar H0 (independentes)"
    return {
        "chi2": chi2,
        "p_value": p,
        "dof": dof,
        "expected_freq": expected,
        "decisão": decisao
    }

def calcular_z_score(serie):
    """
    Calcula o z-score de uma série numérica.
    Retorna uma série com o mesmo índice.
    """
    return pd.Series(_zscore(serie, nan_policy='omit'), index=serie.index)

def teste_t_student(df, coluna_valor, coluna_grupo, grupo1, grupo2, equal_var=True, alpha=0.05):
    """
    Teste t-Student para duas amostras independentes.
    - coluna_valor: coluna numérica
    - coluna_grupo: coluna categórica com os dois grupos
    - grupo1, grupo2: nomes dos grupos a comparar
    Retorna estatística t, p-valor e decisão.
    """
    amostra1 = df[df[coluna_grupo] == grupo1][coluna_valor]
    amostra2 = df[df[coluna_grupo] == grupo2][coluna_valor]
    t_stat, p = ttest_ind(amostra1, amostra2, equal_var=equal_var, nan_policy='omit')
    decisao = "rejeita H0 (médias diferentes)" if p < alpha else "falha em rejeitar H0 (sem diferença)"
    return {
        "t_statistic": t_stat,
        "p_value": p,
        "decisão": decisao
    }